<a href="https://colab.research.google.com/github/taechang1/ai/blob/master/Chapter01_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>